This notebook is used to support the chapter on data splitting.


In [ ]:
using DataFrames
import CSV
using Statistics
using GLM
using CairoMakie
using Markdown
CairoMakie.activate!(; px_per_unit = 2)

We first load the data. The data as presented here have a column for the temperature in March, which has been smoothed, and for the day of bloom, which is expressed as day of year:


In [ ]:
sakura = DataFrame(CSV.File(joinpath(@__DIR__, "..", "data", "general", "sakura.csv")))
allowmissing!(sakura, :temperature)
replace!(sakura.temperature, -50.0 => missing)
sort!(sakura, :year)
describe(sakura)

We will use a 14 years running average for the date of bloom; this is a little artificial, but it 


In [ ]:
average = zeros(Float64, size(sakura, 1))
for (i, row) in enumerate(eachrow(sakura))
    year_span = row.year .+ (-8, 0)
    @info year_span
    valid_records = year_span[1] .< sakura.year .< year_span[2]
    subset = sakura[findall(valid_records), :]
    dropmissing!(subset)
    if size(subset, 1) >= 3
        average[i] = mean(subset.flowering)
    end
end
sakura.flowavg = average

allowmissing!(sakura, :flowavg)
replace!(sakura.flowavg, 0.0 => missing)

dropmissing!(sakura)

We can inspect the data to look for a linear relationship between temperature and flowering time:


In [ ]:
#| label: fig-splits-rawdata
#| fig-cap: 'The raw data show a negative relationship between the temperature in March, and the bloom time. This suggests that when the trees have accumulated enough temperature, they can bloom early. In a context of warming, we should therefore see earlier blooms with rising temperatures.'
f = Figure(; resolution=(500, 300))
ax = Axis(f[1,1]; xlabel="March temperature (°C)", ylabel="Bloom time (day of year)")
scatter!(ax, sakura.temperature, sakura.flowering, color=:lightgrey)
current_figure()

We can now define a few loss functions to compare them later on:


In [ ]:
mse(tr, pr) = mean((tr .- pr).^2.0)
rmse(tr, pr) = sqrt(mse(tr, pr))
mae(tr, pr) = mean(abs.(tr .- pr))
mbe(tr, pr) = mean(tr .- pr)

We now define a series of folds, in order to do k-folds cross validation. Note that we keep a holdout test dataset corresponding to the end of the time series for model evaluation.


In [ ]:
n = size(sakura, 1)
test_size = ceil.(Int, 0.25n)

sakuratrain = sakura[1:(n-test_size), :]
sakuratest = sakura[(n-test_size+1):n, :]

folds = ceil.(Int, LinRange(1, size(sakuratrain, 1), 11))

This next figure is purely an illustration of the splits:


In [ ]:
#| label: fig-splits-illustration
#| fig-cap: 'An illustration of a series of folds on a timeseries. The grey aata are used for training, the black data for validation, and the red data are kept for testing. Note that in this context, we sometimes use the future to validate on the past (look at the first fold!), but this is acceptable for reasons explained in the text.'
f = Figure()
s1 = Axis(f[1,1])
s2 = Axis(f[2,1])
s3 = Axis(f[3,1])
s4 = Axis(f[4,1])
for (i,s) in enumerate([s1, s2, s3, s4])
    scatter!(s, sakuratrain.year, sakuratrain.flowavg, color=:lightgrey)
    scatter!(s, sakuratest.year, sakuratest.flowavg, color=Makie.wong_colors()[4], linewidth=4)
    j = 2i-1
    valid_idx = folds[j]:folds[j+1]
    train_idx = filter(n -> !(n in valid_idx), 1:size(sakuratrain, 1))
    valid = sakuratrain[valid_idx, :]
    train = sakuratrain[train_idx, :]
    scatter!(s, valid.year, valid.flowavg, color=Makie.wong_colors()[3], linewidth=2)
    hidedecorations!(s)
    hidespines!(s)
end
current_figure()

placeholders for validation measures


In [ ]:
test_results = DataFrame()

this is the cross validation


In [ ]:
for (i, Ki) in enumerate(folds)
    if Ki < size(sakuratrain, 1)
        valid_idx = folds[i]:folds[i+1]
        train_idx = filter(n -> !(n in valid_idx), 1:size(sakuratrain, 1))
        valid = sakuratrain[valid_idx, :]
        train = sakuratrain[train_idx, :]
        m1 = lm(@formula(flowavg ~ temperature), train)
        m2 = lm(@formula(flowavg ~ temperature + temperature^-1), train)
        for (md, mn) in [(m1, :Simple), (m2, :Inverse)]
            for (ds, dn) in [(train, :Training), (valid, :Validation)]
                for (m,p) in [(mse, :MSE), (rmse, :RMSE), (mae, :MAE), (mbe, :MBE)]
                    loss = m(ds.flowavg, predict(md, ds))
                    push!(test_results,
                        (
                            Model = mn,
                            Dataset = dn,
                            Measure = p,
                            Value = loss
                        )
                    )
                end
            end
        end
    end
end

gr_test = groupby(test_results, [:Model, :Dataset, :Measure])
benchmark_result = combine(gr_test, :Value => mean => Symbol("Loss (avg.)"), :Value => std => Symbol("Loss (std. dev.)"))
sort!(benchmark_result, [:Measure, :Dataset, :Model, Symbol("Loss (avg.)")])

Based on these results, we can look at the value of the MAE loss to pick our best model:


In [ ]:
choice = benchmark_result[(benchmark_result.Measure .== :MAE),:]
sort!(choice, :Dataset)

It looks like the models with additional terms do not work any better than a simple regression against the temperature, so we will select the simpler model. We now train the model on the full dataset:


In [ ]:
model = lm(@formula(flowavg ~ temperature), sakuratrain)

In [ ]:
#| label: fig-splits-prediction
#| fig-cap: TODO
scatter(sakura.year, sakura.flowavg, color=:lightgrey)
lines!(sakuratrain.year, predict(model, sakuratrain), color=:black)
lines!(sakuratest.year, predict(model, sakuratest), color=:red)
current_figure()

In [ ]:
#| label: fig-splits-validation
#| fig-cap: TODO
f = Figure(; resolution=(500, 300))
ax = Axis(f[1,1]; xgridvisible=false, ygridvisible=false, xlabel="Fold", ylabel="MAE loss")
scatterlines!(ax, train_mae, color=:grey, label="Training data")
scatterlines!(ax, valid_mae, color=:black, label="Validation data")
hlines!(ax, mae(sakuratest.flowavg, predict(model, sakuratest)), color=:red, label="Testing data")
hidespines!(ax, :l, :r, :t, :b)
hidexdecorations!(ax; label=false)
ylims!(ax, (0, 5))
axislegend(ax)
current_figure()

We can now make a table with the results (by dataset/loss function):


In [ ]:
#| label: tab-splits-results
#| tbl-cap: blah
output = DataFrame()
output.Loss = [:RMSE, :MSE, :MAE, :MBE]
output.Training = [
    mean(train_rmse),
    mean(train_mse),
    mean(train_mae),
    mean(train_mbe)
]
output.Validation = [
    mean(valid_rmse),
    mean(valid_mse),
    mean(valid_mae),
    mean(valid_mbe)
]
output.Testing = [
    rmse(sakuratest.flowavg, predict(model, sakuratest)),
    mse(sakuratest.flowavg, predict(model, sakuratest)),
    mae(sakuratest.flowavg, predict(model, sakuratest)),
    mbe(sakuratest.flowavg, predict(model, sakuratest))
]
print(output)

In [ ]:
#| label: fig-splits-detail
#| fig-cap: TODO
scatter(sakuratest.year, sakuratest.flowavg, color=:lightgrey)
lines!(sakuratest.year, predict(model, sakuratest), color=:red)
current_figure()